In [96]:
import numpy as np 
import pandas as pd
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import optuna
from category_encoders import OneHotEncoder, MEstimateEncoder, CatBoostEncoder, OrdinalEncoder
from sklearn import set_config
import category_encoders
from sklearn.inspection import permutation_importance, PartialDependenceDisplay
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, train_test_split
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestRegressor, IsolationForest
from sklearn.metrics import roc_auc_score, roc_curve, make_scorer, f1_score, accuracy_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer, KNNImputer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin, clone
from sklearn.preprocessing import FunctionTransformer,StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.preprocessing import PolynomialFeatures
import category_encoders as ce
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.ensemble import VotingClassifier, StackingClassifier
from sklearn.svm import SVC
from sklearn.metrics import auc, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.gaussian_process import GaussianProcessClassifier
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import squareform
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier, Pool
import warnings
from great_tables import GT, style ,exibble, from_column, loc
from colorama import Style, Fore

sns.set_theme(style = 'white', palette = 'colorblind')
pal = sns.color_palette('colorblind')

pd.set_option('display.max_rows', 100)
set_config(transform_output = 'pandas')
pd.options.mode.chained_assignment = None
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.set_option('display.max_rows', 150)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [34]:
def customStatistic(df: pd.DataFrame(), categoric = False):
    num_cols = list(df._get_numeric_data())
    cat_cols = list(df.drop(num_cols,axis=1))
    if categoric:
        desc = pd.DataFrame(index = list(df[cat_cols]))
        df = df[cat_cols]
    else:
        desc = pd.DataFrame(index = list(df[num_cols]))
        df = df[num_cols]
        desc['skew'] = df[num_cols].skew()
        
    desc['type'] = df.dtypes
    desc['count'] = df.count()
    desc['nunique'] = df.nunique()
    desc['%unique'] = desc['nunique'] /len(df) * 100 
    desc['null'] = df.isnull().sum()
    desc['%null'] = desc['null'] / len(df) * 100
    desc = pd.concat([desc,df.describe().T.drop('count',axis=1)],axis=1)    

    desc = desc.round(2)
    return desc.reset_index().rename(columns={'index':'Column'}).sort_values(by=['type'])

In [35]:
train = pd.read_csv(r'data/train.csv', index_col='id')
test = pd.read_csv(r'data/test.csv', index_col='id')
sub  = pd.read_csv(r'data/sample_submission.csv')

In [36]:
target_categories = ['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']
idx_vals = np.argmax(train[target_categories], axis = 1)

target_vals = []
for i in range(0, len(idx_vals)):
    target_vals.append(target_categories[idx_vals[i]])

train['TARGET'] = target_vals
train = train.drop(target_categories, axis = 1)

In [37]:
stat = customStatistic(train,False)
GT(stat)\
    .tab_header(title='Descriptive Statistic - Train', subtitle='Numeric Fields')\
    .data_color(columns=['min','max','mean'],palette=['lightblue','lightcoral'],alpha=0.5)\
    .fmt_percent(columns=['%unique','%null'])

GT(_tbl_data=                   Column  skew     type  count  nunique  %unique  null  \
0               X_Minimum  0.02    int64  19219     1191     6.20     0   
12       TypeOfSteel_A400 -0.39    int64  19219        2     0.01     0   
11       TypeOfSteel_A300  0.40    int64  19219        2     0.01     0   
10     Length_of_Conveyer  0.86    int64  19219       99     0.52     0   
9   Maximum_of_Luminosity  1.17    int64  19219       98     0.51     0   
8   Minimum_of_Luminosity -0.33    int64  19219      162     0.84     0   
7       Sum_of_Luminosity  6.69    int64  19219     2595    13.50     0   
13  Steel_Plate_Thickness  2.36    int64  19219       27     0.14     0   
5             X_Perimeter  6.32    int64  19219      460     2.39     0   
4            Pixels_Areas  6.98    int64  19219     1154     6.00     0   
3               Y_Maximum  2.57    int64  19219     3341    17.38     0   
2               Y_Minimum  2.58    int64  19219     3345    17.40     0   
1               X_Maximum  0.07    int64  19219     1259     6.55     0   
6             Y_Perimeter  2.98    int64  19219      331     1.72     0   
24      Orientation_Index -0.19  float64  19219     1598     8.31     0   
23            Log_Y_Index  0.39  float64  19219      241     1.25     0   
22            Log_X_Index  1.10  float64  19219      206     1.07     0   
21             LogOfAreas  0.86  float64  19219     1072     5.58     0   
20   Outside_Global_Index -0.37  float64  19219        4     0.02     0   
17        Outside_X_Index  3.12  float64  19219      525     2.73     0   
18          Edges_X_Index -0.25  float64  19219     1102     5.73     0   
16           Square_Index  0.01  float64  19219     1118     5.82     0   
15            Empty_Index  0.20  float64  19219     1748     9.10     0   
14            Edges_Index  0.54  float64  19219     1849     9.62     0   
25       Luminosity_Index  0.78  float64  19219     2046    10.65     0   
19          Edges_Y_Index -0.98  float64  19219      900     4.68     0   
26         SigmoidOfAreas  0.22  float64  19219      467     2.43     0   

    %null        mean         std      min        25%         50%         75%  \
0     0.0      709.85      531.54     0.00      49.00      777.00     1152.00   
12    0.0        0.60        0.49     0.00       0.00        1.00        1.00   
11    0.0        0.40        0.49     0.00       0.00        0.00        1.00   
10    0.0     1459.35      145.57  1227.00    1358.00     1364.00     1652.00   
9     0.0      128.65       14.20    39.00     124.00      127.00      135.00   
8     0.0       84.81       28.80     0.00      70.00       90.00      105.00   
7     0.0   191846.68   442024.69   250.00    9848.00    18238.00    67978.00   
13    0.0       76.21       53.93    40.00      40.00       69.00       80.00   
5     0.0       95.65      177.82     2.00      15.00       25.00       64.00   
4     0.0     1683.99     3730.32     6.00      89.00      168.00      653.00   
3     0.0  1846605.35  1896295.14  6724.00  657502.00  1398179.00  2362511.00   
2     0.0  1849756.04  1903553.85  6712.00  657468.00  1398169.00  2368032.00   
1     0.0      753.86      499.84     4.00     214.00      796.00     1165.00   
6     0.0       64.12      101.05     1.00      14.00       23.00       61.00   
24    0.0        0.10        0.49    -0.99      -0.27        0.11        0.53   
23    0.0        1.39        0.41     0.00       1.08        1.32        1.71   
22    0.0        1.31        0.47     0.30       1.00        1.15        1.43   
21    0.0        2.47        0.76     0.78       1.95        2.23        2.81   
20    0.0        0.59        0.48     0.00       0.00        1.00        1.00   
17    0.0        0.03        0.05     0.00       0.01        0.01        0.02   
18    0.0        0.61        0.22     0.01       0.45        0.64        0.79   
16    0.0        0.57        0.26     0.01       0.38        0.55        0.82   
15    0.0        0.41  

In [38]:
stat = customStatistic(test,False)
GT(stat)\
    .tab_header(title='Descriptive Statistic - Test', subtitle='Numeric Fields')\
    .data_color(columns=['min','max','mean'],palette=['lightblue','lightcoral'],alpha=0.5)\
    .fmt_percent(columns=['%unique','%null'])

GT(_tbl_data=                   Column   skew     type  count  nunique  %unique  null  \
0               X_Minimum   0.01    int64  12814     1131     8.83     0   
12       TypeOfSteel_A400  -0.38    int64  12814        2     0.02     0   
11       TypeOfSteel_A300   0.39    int64  12814        2     0.02     0   
10     Length_of_Conveyer   0.85    int64  12814       92     0.72     0   
9   Maximum_of_Luminosity   1.36    int64  12814       99     0.77     0   
8   Minimum_of_Luminosity  -0.33    int64  12814      161     1.26     0   
7       Sum_of_Luminosity   9.06    int64  12814     2300    17.95     0   
13  Steel_Plate_Thickness   2.40    int64  12814       24     0.19     0   
5             X_Perimeter  19.42    int64  12814      435     3.39     0   
4            Pixels_Areas   7.60    int64  12814     1070     8.35     0   
3               Y_Maximum   2.56    int64  12814     2827    22.06     0   
2               Y_Minimum   2.56    int64  12814     2781    21.70     0   
1               X_Maximum   0.06    int64  12814     1188     9.27     0   
6             Y_Perimeter  69.40    int64  12814      316     2.47     0   
24      Orientation_Index  -0.18  float64  12814     1382    10.79     0   
23            Log_Y_Index   0.42  float64  12814      218     1.70     0   
22            Log_X_Index   1.11  float64  12814      189     1.47     0   
21             LogOfAreas   0.85  float64  12814      994     7.76     0   
20   Outside_Global_Index  -0.34  float64  12814        3     0.02     0   
17        Outside_X_Index   3.63  float64  12814      480     3.75     0   
18          Edges_X_Index  -0.26  float64  12814      998     7.79     0   
16           Square_Index   0.03  float64  12814      987     7.70     0   
15            Empty_Index   0.23  float64  12814     1606    12.53     0   
14            Edges_Index   0.53  float64  12814     1697    13.24     0   
25       Luminosity_Index   0.63  float64  12814     1887    14.73     0   
19          Edges_Y_Index  -1.00  float64  12814      788     6.15     0   
26         SigmoidOfAreas   0.22  float64  12814      427     3.33     0   

    %null        mean         std      min        25%         50%         75%  \
0     0.0      709.33      531.88     0.00      46.00      776.00     1152.00   
12    0.0        0.59        0.49     0.00       0.00        1.00        1.00   
11    0.0        0.41        0.49     0.00       0.00        0.00        1.00   
10    0.0     1460.14      146.11  1227.00    1358.00     1364.00     1652.00   
9     0.0      128.69       14.28    37.00     124.00      127.00      135.00   
8     0.0       84.55       28.71     0.00      68.00       90.00      105.00   
7     0.0   192689.30   481287.15   535.00    9968.00    18402.00    66960.25   
13    0.0       75.87       53.49    40.00      40.00       69.00       80.00   
5     0.0       96.46      219.93     2.00      15.00       24.00       64.00   
4     0.0     1672.91     3725.52     4.00      91.00      169.00      649.00   
3     0.0  1842029.49  1911425.61  6724.00  639857.00  1378588.00  2356237.50   
2     0.0  1846533.46  1916427.39  6712.00  640173.75  1378890.50  2368032.00   
1     0.0      753.27      500.74     4.00     214.00      796.00     1167.00   
6     0.0       64.73      188.51     2.00      14.00       24.00       60.00   
24    0.0        0.10        0.49    -0.99      -0.28        0.11        0.53   
23    0.0        1.39        0.40     0.30       1.08        1.32        1.71   
22    0.0        1.31        0.47     0.34       1.00        1.15        1.43   
21    0.0        2.47        0.76     0.78       1.96        2.23        2.81   
20    0.0        0.58        0.48     0.00       0.00        1.00        1.00   
17    0.0        0.03        0.05     0.00       0.01        0.01        0.02   
18    0.0        0.62        0.22     0.01       0.46        0.64        0.79   
16    0.0        0.57        0.26     0.01       0.38        0.54        0.8

In [39]:
def plot_numerical():
    #num = train.select_dtypes(include=['int64','float64']).columns

    df = pd.concat([train[NUMERIC_COLS].assign(Source = 'Train'), 
                    test[NUMERIC_COLS].assign(Source = 'Test')], ignore_index = True)

    # Use of more advanced artistic matplotlib interface (see the axes)
    fig, axes = plt.subplots(len(NUMERIC_COLS), 3 ,figsize = (16, len(NUMERIC_COLS) * 4), 
                             gridspec_kw = {'hspace': 0.35, 'wspace': 0.3, 
                                            'width_ratios': [0.80, 0.20, 0.20]})

    for i,col in enumerate(NUMERIC_COLS):
        ax = axes[i,0]
        sns.kdeplot(data = df[[col, 'Source']], x = col, hue = 'Source', palette=['#456cf0', '#ed7647'], linewidth = 2.1, warn_singular=False, ax = ax) # Use of seaborn with artistic interface
        ax.set_title(f"\n{col}",fontsize = 9)
        ax.grid(visible=True, which = 'both', linestyle = '--', color='lightgrey', linewidth = 0.75)
        ax.set(xlabel = '', ylabel = '')

        ax = axes[i,1]
        sns.boxplot(data = df.loc[df.Source == 'Train', [col]], y = col, width = 0.25, linewidth = 0.90, fliersize= 2.25, color = '#456cf0', ax = ax)
        ax.set(xlabel = '', ylabel = '')
        ax.set_title("Train", fontsize = 9)

        ax = axes[i,2]
        sns.boxplot(data = df.loc[df.Source == 'Test', [col]], y = col, width = 0.25, linewidth = 0.90, fliersize= 2.25, color = '#ed7647', ax = ax)
        ax.set(xlabel = '', ylabel = '')
        ax.set_title("Test", fontsize = 9)

    plt.suptitle(f'\nDistribution analysis - numerical features',fontsize = 12, y = 0.89, x = 0.57, fontweight='bold')
    plt.show()

In [51]:
NUMERIC_COLS = train.select_dtypes(exclude='object').columns

# plot_numerical()

In [ ]:
# Set number of rows and columns for subplots
num_rows = 6
num_cols = 3

# Create subplots
fig, axes = plt.subplots(num_rows, num_cols, figsize=(30, 30))

# Flatten axes array for easy iteration
axes = axes.flatten()

for i, feature in enumerate(features):
    # Create a new column to mark labels
    df_train['classes'] = train[labels].idxmax(axis=1)
    
    # Set order of hue for consistency
    hue_order = ['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']
    
    # Plot histogram with marked labels
    sns.histplot(data=train, x=feature, hue='classes', multiple='stack', hue_order=hue_order, legend=True, ax=axes[i])
    
    # Set title and labels
    axes[i].set_title(f'Distribution of {feature} with classes')
    axes[i].set_xlabel(feature)
    axes[i].set_ylabel('Count')

# Adjust layout to prevent overlapping of subplots
plt.tight_layout()

# Show the plots
plt.show()

# Drop the temporary column
df_train.drop('classes', axis=1, inplace=True)

# Model Building

In [92]:
target_variables = ['Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults']

In [99]:
train = pd.read_csv(r'data/train.csv', index_col='id')
test = pd.read_csv(r'data/test.csv', index_col='id')
df_submission = pd.read_csv('data/sample_submission.csv')


X = train.drop(target_variables, axis=1)
y = train[target_variables]

# Train Test Split
X_train , X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [102]:
# Define XGBoost parameters
xgboost_params = {
    "objective": "binary:logistic",
    "eval_metric": "auc",
    #"max_depth": 12,
    "learning_rate": 0.005,
    "subsample": 0.85,
    "colsample_bytree": 0.85,
    "alpha": 0.001,
    "lambda": 0.001,
    "gamma": 0.1,
    "min_child_weight": 5,
    "n_estimators": 1500,
    "random_state": 42
}
df_submission[target_variables] = 0
# Initialize XGBoost classifier
xgboost_model = XGBClassifier(**xgboost_params)

# List to store AUC scores for each target variable
auc_scores = []

# Perform cross-validation for each target variable
for target in target_variables:
    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, y[target], test_size=0.2, random_state=42)
    
    # Fit the model
    xgboost_model.fit(X_train, y_train)
    
    # Predict probabilities
    y_pred_proba = xgboost_model.predict_proba(X_test)[:, 1]
    
    # Calculate AUC score
    auc_score = roc_auc_score(y_test, y_pred_proba)
    auc_scores.append(auc_score)

    # Add predictions to submission DataFrame
    df_submission[target] += xgboost_model.predict_proba(test)[:, 1]

# Calculate mean AUC score
mean_auc_score = np.mean(auc_scores)

# Print mean AUC score
print("Mean AUC Score (XGBoost):", mean_auc_score)

Mean AUC Score (XGBoost): 0.8882271738632525


In [103]:
# Save submission file
df_submission.to_csv('submission.csv', index=False)